第四届魔镜杯大赛：
本赛题对回款预测问题进行了简化，选手需要分别预测每个资产标的第一期从成交日期至第一期::::应还款日期::::每日的还款金额，并最终在整体上以资产组合每日还款的误差作为评价指标。

1. 数据集组成：
train.csv 和 test.csv  其中train.csv 包含所有features 而test.csv不包含repay_amt 并且train.csv中用户可能出现多次数据信息，需要进行处理，因为test.csv中只存在一条用户信息
![image.png](attachment:image.png)

In [3]:
import pandas as pd
import numpy as np
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")
train_data.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,748147,3163926,2018-04-25,2018-05-25,72.1167,2018-05-25,72.1167
1,672952,3698760,2018-06-09,2018-07-09,258.7045,2018-07-08,258.7045
2,404196,2355665,2018-02-18,2018-03-18,307.9270,\N,\N
3,342769,1994522,2018-01-13,2018-02-13,252.9809,2018-02-13,252.9809
4,828139,3602352,2018-06-01,2018-07-01,107.6503,2018-06-25,107.6503


In [26]:
test_data.head()

,user_id,listing_id,auditing_date,due_date,due_amt
0,498765,5431438,2019-03-12,2019-04-12,138.5903
1,34524,5443211,2019-03-15,2019-04-15,208.0805
2,821741,5461707,2019-03-22,2019-04-22,421.2097
3,263534,5472320,2019-03-26,2019-04-26,212.6537
4,238853,5459750,2019-03-21,2019-04-21,817.4593


In [27]:
print(train_data.shape[0])#重复用户
print(test_data.shape[0])

1000000
130000


2. 标的属性表（listing_info.csv）
标的属性表包含了本赛题涉及的所有标的，包括：(1) 样本集中所有标的；(2) 样本集中所有借款用户在过去一段时间内成交和还款的标的。标的属性信息在成交时确定，后续不再变更。
![image.png](attachment:image.png)
注：上述的数据都经过脱敏处理，脱敏后的数据分布和数值大小关系与脱敏前基本一致。

In [28]:
list_info = pd.read_csv("./data/listing_info.csv")
list_info.head()

,user_id,listing_id,auditing_date,term,rate,principal
0,316610,1556649,2017-11-26,9,7.6,4800
1,62002,1556633,2017-11-26,6,7.6,4000
2,192135,1556629,2017-11-26,12,8.0,8660
3,487382,1556628,2017-11-26,9,7.6,4780
4,235186,1556627,2017-11-26,9,7.6,1480


In [29]:
list_info.shape[0]#存在多个同一用户的不同数据

5484891

3. 借款用户基础信息表（user_info.csv）
借款用户基础信息表包含了本赛题涉及的所有用户，用户信息可能发生变更，表中同一用户可能存在多条数据。
![image.png](attachment:image.png)
如何处理同一用户的多条数据？？？

In [30]:
user_info = pd.read_csv("./data/user_info.csv")
user_info.head()

,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
0,483833,2017-04,男,19,c29,c26,c26241,2018-12-11
1,156772,2016-05,男,31,c11,c11,c11159,2018-02-13
2,173388,2016-05,男,34,c02,c02,c02182,2018-08-21
3,199107,2016-07,女,25,c09,c09,c09046,2018-06-05
4,122560,2016-03,男,23,c05,c05,c05193,2018-04-02


In [31]:
user_info.shape[0]

954209

4. 用户画像标签列表（user_taglist.csv）
用户画像标签列表提供了用户的标签信息，用户标签可能发生变更，表中同一用户可能存在多条数据；若在表中无法查到用户标签信息，则表示该用户标签信息未知。
![image.png](attachment:image.png)

In [32]:
user_taglist = pd.read_csv("./data/user_taglist.csv")
user_taglist.head()

,user_id,taglist,insertdate
0,113401,4707|473|3498|4759|1654|298|2869|1164|212|1885...,2018-10-03
1,378358,751|2207|1100|2099|1832|1911|5347|2254|171|360...,2018-11-30
2,434838,877|3795|5628|70|2684|691|719|4228|631|1541|12...,2018-03-25
3,577061,2431|3242|340|1823|4020|4357|164|620|2168|1192...,2018-05-25
4,566753,3980|3125|1819|1333|1177|3972|621|5800|3632|16...,2018-12-02


In [35]:
user_taglist.shape[0]

615160

5. 借款用户操作行为日志表（user_behavior_logs.csv）
借款用户操作行为日志表提供了每位用户在过去一段时间内的操作行为日志数据，行为发生时间精确到秒级，相同用户的相同行为在同一秒内可能出现多条数据。
![image.png](attachment:image.png)

In [36]:
user_behavior_logs = pd.read_csv("./data/user_behavior_logs.csv")
user_behavior_logs.head()

,user_id,behavior_time,behavior_type
0,842439,2018-09-13 23:17:21,3
1,842439,2018-09-13 23:17:21,3
2,905214,2018-09-13 15:19:30,3
3,905214,2018-09-13 15:19:30,3
4,842439,2018-09-13 23:17:21,3


In [37]:
user_behavior_logs.shape[0]

55781271

6. 用户还款日志表（user_repay_logs.csv）
借款用户还款日志表提供了每位用户在过去一段时期内的还款日志数据。
![image.png](attachment:image.png)

In [4]:
user_repay_logs = pd.read_csv("./data/user_repay_logs.csv")
user_repay_logs.head(20)

,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt
0,748483,1858122,6,2018-06-29,528.6365,2018-06-20,528.6365
1,748483,1858122,4,2018-04-29,528.6365,2200-01-01,528.6365
2,748483,1858122,7,2018-07-29,528.6365,2018-06-20,528.6365
3,748483,1858122,5,2018-05-29,528.6365,2018-05-29,528.6365
4,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365
5,748483,1858122,2,2018-02-28,528.6365,2018-02-16,528.6365
6,748483,1858122,3,2018-03-29,528.6365,2018-03-28,528.6365
7,445749,185013,11,2018-03-05,197.9548,2017-05-25,197.9548
8,445749,185013,6,2017-10-05,197.9548,2017-05-25,197.9548
9,445749,185013,10,2018-02-05,197.9548,2017-05-25,197.9548


In [39]:
user_repay_logs.shape[0]

18001297

思路： 
1.首先做一个合并表，将所有特征按照["user_id","listing_id","reg_mon","gender","age","cell_province","id_province","id_city","taglist","behavior_type","behavior_time","auditing_date","term","rate","principal","order_id","due_date","due_amt","repay_date","repay_amt"]
2.可视化操作，比较各个特征和结果之间的相关性  
3.特征工程（考虑组合新的特征）   
4.评价指标    
5.选择合适的算法进行训练

In [40]:
#取出无重复的user_id
user_id = test_data["user_id"]
#user_id

In [1]:
#合并train_data和user_info
#new_data1 = []
#train_data = pd.DataFrame()
#new_data1 = pd.merge(train_data,user_info,how = "train_data",on = "user_id")
#new_data1